In [ ]:
import math

import torch
import torch.nn as nn
import torch.nn.functional as F

import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.gridspec as gridspec
import scipy.io
from scipy.interpolate import griddata

!pip install pyDOE
from pyDOE import lhs

from collections import OrderedDict
from tqdm import tqdm

sns.set_style("white")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pyDOE: filename=pyDOE-0.3.8-py3-none-any.whl size=18184 sha256=b5e31ee2cb7bc3b361d3c0e2988ebb668a310f073803cf3105b081381ef4d0d0
  Stored in directory: /root/.cache/pip/wheels/00/3e/fa/5705bf59c2053c17c4799c3ab66a2e356c32f40a3044fe2134
Successfully built pyDOE


In [ ]:
# Multi-layer Perceptron (Deep Neural Network)
class DNN(torch.nn.Module):
    def __init__(self, layers):
        super(DNN, self).__init__()
        
        self.depth = len(layers) - 1
        self.activation = torch.nn.Tanh
        
        layer_list = list()
        for i in range(self.depth - 1): 
            layer_list.append(('layer_%d' % i, torch.nn.Linear(layers[i], layers[i+1])))
            layer_list.append(('activation_%d' % i, self.activation()))
            
        layer_list.append(('layer_%d' % (self.depth - 1), torch.nn.Linear(layers[-2], layers[-1])))
        layerDict = OrderedDict(layer_list)
        
        self.layers = torch.nn.Sequential(layerDict)
        
    def forward(self, x):
        out = self.layers(x)
        return out

In [ ]:
# Physics Informed Neural Network
class PINN():
    def __init__(self, X_s, s, X_f, layers, lb, ub, nu):
        device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

        # boundary conditions
        self.lb = torch.tensor(lb).float().to(device)
        self.ub = torch.tensor(ub).float().to(device)
        
        # data
        self.x_s = torch.tensor(X_s[:, 0:1], requires_grad=True).float().to(device)
        self.y_s = torch.tensor(X_s[:, 1:2], requires_grad=True).float().to(device)
        self.x_f = torch.tensor(X_f[:, 0:1], requires_grad=True).float().to(device)
        self.y_f = torch.tensor(X_f[:, 1:2], requires_grad=True).float().to(device)
        self.s = torch.tensor(s).float().to(device)
        
        # deep neural networks
        self.layers = layers
        self.nu = nu
        self.dnn = DNN(layers).to(device)
        
        # optimizers: using the same settings
        self.optimizer = torch.optim.LBFGS(
            self.dnn.parameters(), 
            lr=1.0, 
            max_iter=50000, 
            max_eval=50000, 
            history_size=50,
            tolerance_grad=1e-5, 
            tolerance_change=1.0 * np.finfo(float).eps,
            line_search_fn="strong_wolfe"      
        )
        self.iter = 0

    def net_s(self, x, y):  
        s = self.dnn(torch.cat([x, y], dim=1))
        return s
    
    def net_f(self, x, y):
        s = self.net_s(x, y)
        u = s[:,0]
        v = s[:,1]
        p = s[:,2]

        du_dx = torch.autograd.grad(inputs=x, outputs=u, grad_outputs=torch.ones_like(u), retain_graph=True, create_graph=True)[0]
        dv_dx = torch.autograd.grad(inputs=x, outputs=v, grad_outputs=torch.ones_like(v), retain_graph=True, create_graph=True)[0]
        dp_dx = torch.autograd.grad(inputs=x, outputs=p, grad_outputs=torch.ones_like(p), retain_graph=True, create_graph=True)[0]
        
        du_dy = torch.autograd.grad(inputs=y, outputs=u, grad_outputs=torch.ones_like(u), retain_graph=True, create_graph=True)[0]
        dv_dy = torch.autograd.grad(inputs=y, outputs=v, grad_outputs=torch.ones_like(v), retain_graph=True, create_graph=True)[0]
        dp_dy = torch.autograd.grad(inputs=y, outputs=p, grad_outputs=torch.ones_like(p), retain_graph=True, create_graph=True)[0]

        du_dxx = torch.autograd.grad(inputs=x, outputs=du_dx, grad_outputs=torch.ones_like(du_dx), retain_graph=True, create_graph=True)[0]
        du_dyy = torch.autograd.grad(inputs=y, outputs=du_dy, grad_outputs=torch.ones_like(du_dy), retain_graph=True, create_graph=True)[0]
        dv_dxx = torch.autograd.grad(inputs=x, outputs=dv_dx, grad_outputs=torch.ones_like(dv_dx), retain_graph=True, create_graph=True)[0]
        dv_dyy = torch.autograd.grad(inputs=y, outputs=dv_dy, grad_outputs=torch.ones_like(dv_dy), retain_graph=True, create_graph=True)[0]
        
        f1 = u.squeeze() * du_dx + v.squeeze() * du_dy + dp_dx - self.nu * (du_dxx + du_dyy)
        f2 = u.squeeze() * dv_dx + v.squeeze() * dv_dy + dp_dy - self.nu * (dv_dxx + dv_dyy)
        f = f1 + f2
        return f
    
    def loss_func(self):
        self.optimizer.zero_grad()
        
        s_pred = self.net_s(self.x_s, self.y_s)
        f_pred = self.net_f(self.x_f, self.y_f)
#         print("s:", type(self.s))
#         print("f_pred:", type(f_pred))
#         print("s_pred:", type(s_pred))
        loss_s = torch.mean((self.s - s_pred) ** 2)
        loss_f = torch.mean(f_pred ** 2)
        
        loss = loss_s + loss_f
        
        loss.backward()
        self.iter += 1
        if self.iter % 100 == 0:
            print('Iter %d, Loss: %.5e, Loss_u: %.5e, Loss_f: %.5e' % (self.iter, loss.item(), loss_s.item(), loss_f.item()))
        return loss
    
    def train(self):
        self.dnn.train()
        self.optimizer.step(self.loss_func)
     
    def predict(self, X):
        device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
        
        x = torch.tensor(X[:, 0:1], requires_grad=True).float().to(device)
        y = torch.tensor(X[:, 1:2], requires_grad=True).float().to(device)

        self.dnn.eval()
        s = self.net_s(x, y)
        f = self.net_f(x, y)
        s = s.detach().cpu().numpy()
        f = f.detach().cpu().numpy()
        return s, f

In [ ]:
N_u = 200
N_f = 10000

Re = 20
nu = 1 / Re
lamb = 1 / (2 * nu) - np.sqrt(1 / (4 * nu ** 2) + 4 * np.pi ** 2)

layers = [2, 50, 50, 50, 50, 3]

h = 0.01
k = 0.01
x = torch.arange(0, 1, h)
y = torch.arange(0, 1, k)

print("x:", x.shape)
print("y:", y.shape)

X, Y = np.meshgrid(x,y)
X = torch.from_numpy(X)
Y = torch.from_numpy(Y)

print("X:", X.shape)
print("Y:", Y.shape)

Exact = torch.zeros(X.shape[0],X.shape[1],3)
Exact_U = 1-(torch.exp(lamb*X))*(torch.cos(2*math.pi*Y))
Exact_V = (lamb/(2*math.pi))*(torch.exp(lamb*X))*(torch.sin(2*math.pi*Y))
Exact_P = (1/2)*(1-(torch.exp(2*lamb*X)))
Exact[:,:,0] = Exact_U
Exact[:,:,1] = Exact_V
Exact[:,:,2] = Exact_P

print("Exact:",Exact.shape)

X_test = np.hstack((X.flatten()[:,None], Y.flatten()[:,None]))
s_test = torch.reshape(Exact, (-1, 3))             

print("X_test:", X_test.shape)
print("s_test:", s_test.shape)

# Domain bounds
lb = X_test.min(0)
ub = X_test.max(0) 


idx = np.random.choice(1000, N_u, replace=False)
X_s_train = X_test[idx, :]
s_train   = s_test[idx, :]

print("X_s_train:", X_s_train.shape)
print("s_train:", s_train.shape)

X_f_train = lb + (ub-lb)*lhs(2, N_f)
X_f_train = np.vstack((X_f_train, X_s_train))

print("X_f_train:", X_f_train.shape)

x: torch.Size([100])
y: torch.Size([100])
X: torch.Size([100, 100])
Y: torch.Size([100, 100])
Exact: torch.Size([100, 100, 3])
X_test: (10000, 2)
s_test: torch.Size([10000, 3])
X_s_train: (200, 2)
s_train: torch.Size([200, 3])
X_f_train: (10200, 2)


In [ ]:
model = PINN(X_s_train, s_train, X_f_train, layers, lb, ub, nu)
model.train()

<ipython-input-3-b422b31a158f>:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.s = torch.tensor(s).float().to(device)


Iter 100, Loss: 3.38184e-03, Loss_u: 2.48239e-03, Loss_f: 8.99448e-04
Iter 200, Loss: 2.19783e-03, Loss_u: 1.27746e-03, Loss_f: 9.20368e-04
Iter 300, Loss: 1.88676e-03, Loss_u: 9.09894e-04, Loss_f: 9.76867e-04


In [ ]:
s_pred, f_pred = model.predict(X_test)

# Relative L2 Error
error_s = np.linalg.norm(s_test-s_pred,2)/np.linalg.norm(s_test,2)
print('Relative L2 Error y: %e' % (error_s))

# Absolute error
u_pred = griddata(X_test, s_pred[:,0].flatten(), (X, Y), method='cubic')
v_pred = griddata(X_test, s_pred[:,1].flatten(), (X, Y), method='cubic')
p_pred = griddata(X_test, s_pred[:,2].flatten(), (X, Y), method='cubic')

Error = np.abs(Exact[:,:,0] - u_pred) + np.abs(Exact[:,:,1] - v_pred) + np.abs(Exact[:,:,2] - p_pred)

In [ ]:
fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(111)

h = ax.imshow(u_pred.T, interpolation='nearest', cmap='rainbow', 
              extent=[y.min(), y.max(), x.min(), x.max()], 
              origin='lower', aspect='auto')
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.10)
cbar = fig.colorbar(h, cax=cax)
cbar.ax.tick_params(labelsize=15) 

ax.plot(
    X_s_train[:,1], 
    X_s_train[:,0], 
    'kx', label = 'Data (%d points)' % (s_train[:,0].shape[0]), 
    markersize = 4,  # marker size doubled
    clip_on = False,
    alpha=1.0
)

line = np.linspace(x.min(), x.max(), 2)[:,None]
ax.plot(y[25]*np.ones((2,1)), line, 'w-', linewidth = 1)
ax.plot(y[50]*np.ones((2,1)), line, 'w-', linewidth = 1)
ax.plot(y[75]*np.ones((2,1)), line, 'w-', linewidth = 1)

ax.set_xlabel('$x$', size=20)
ax.set_ylabel('$y$', size=20)
ax.legend(
    loc='upper center', 
    bbox_to_anchor=(0.9, -0.05), 
    ncol=5, 
    frameon=False, 
    prop={'size': 15}
)
ax.set_title('$u(x,y)$', fontsize = 20) # font size doubled
ax.tick_params(labelsize=15)

plt.show()

In [ ]:
fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(111)

h = ax.imshow(v_pred.T, interpolation='nearest', cmap='rainbow', 
              extent=[y.min(), y.max(), x.min(), x.max()], 
              origin='lower', aspect='auto')
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.10)
cbar = fig.colorbar(h, cax=cax)
cbar.ax.tick_params(labelsize=15) 

ax.plot(
    X_s_train[:,1], 
    X_s_train[:,0], 
    'kx', label = 'Data (%d points)' % (s_train[:,0].shape[0]), 
    markersize = 4,  # marker size doubled
    clip_on = False,
    alpha=1.0
)

line = np.linspace(x.min(), x.max(), 2)[:,None]
ax.plot(y[25]*np.ones((2,1)), line, 'w-', linewidth = 1)
ax.plot(y[50]*np.ones((2,1)), line, 'w-', linewidth = 1)
ax.plot(y[75]*np.ones((2,1)), line, 'w-', linewidth = 1)

ax.set_xlabel('$x$', size=20)
ax.set_ylabel('$y$', size=20)
ax.legend(
    loc='upper center', 
    bbox_to_anchor=(0.9, -0.05), 
    ncol=5, 
    frameon=False, 
    prop={'size': 15}
)
ax.set_title('$v(x,y)$', fontsize = 20) # font size doubled
ax.tick_params(labelsize=15)

plt.show()

In [ ]:
fig = plt.figure(figsize=(7, 7))
ax = fig.add_subplot(111)

h = ax.imshow(p_pred.T, interpolation='nearest', cmap='rainbow', 
              extent=[y.min(), y.max(), x.min(), x.max()], 
              origin='lower', aspect='auto')
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.10)
cbar = fig.colorbar(h, cax=cax)
cbar.ax.tick_params(labelsize=15) 

ax.plot(
    X_s_train[:,1], 
    X_s_train[:,0], 
    'kx', label = 'Data (%d points)' % (s_train[:,0].shape[0]), 
    markersize = 4,  # marker size doubled
    clip_on = False,
    alpha=1.0
)

line = np.linspace(x.min(), x.max(), 2)[:,None]
ax.plot(y[25]*np.ones((2,1)), line, 'w-', linewidth = 1)
ax.plot(y[50]*np.ones((2,1)), line, 'w-', linewidth = 1)
ax.plot(y[75]*np.ones((2,1)), line, 'w-', linewidth = 1)

ax.set_xlabel('$x$', size=20)
ax.set_ylabel('$y$', size=20)
ax.legend(
    loc='upper center', 
    bbox_to_anchor=(0.9, -0.05), 
    ncol=5, 
    frameon=False, 
    prop={'size': 15}
)
ax.set_title('$p(x,y)$', fontsize = 20) # font size doubled
ax.tick_params(labelsize=15)

plt.show()